# Final Project - Group 13
## Project Title: How Covid-19 Policy Influences Violent Crimes 
### People: Zhiwhen Zhu, Hieu Han Nguyen, Gracie Siu 
### Dataset:
- https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
- https://statepolicies.com/data/library/
- https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/explorer/crime/crime-trend  
- https://apps.bea.gov/regional/histdata/releases/1222gdpstate/index.cfm

### Part 2: Data Cleanning and Data Merging

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gpd
from shapely.geometry import box
%matplotlib inline

## 1. Clean Data
- FBI Crime Relational Data
- COVID-19 Stay-At-Home Order Data
- Socioeconomic Data: GDP Data and Personal Income Data

### FBI Crime Relational Data

In [2]:
relationships = pd.read_csv('relationship_2018-2022.csv')
#Rename columns to be more clear
relationships = relationships.rename(columns={"data_year" : "Year",'ori_first_two':'Abbreviation'})
relationships.sample(5)

,Abbreviation,Year,Acquaintance,Babysittee,Boyfriend/Girlfriend,Child of Boyfriend/Girlfriend,Child,Employee,Employer,Friend,...,Relationship Unknown,Sibling,Stepchild,Spouse,Stepparent,Stepsibling,Stranger,Offender,Ex Spouse,Common Law Spouse
113,MN,2021,9125,63,8606,324,2049,131,82,864,...,12556,1482,237,2345,149,87,6102,2537,459,104
51,HI,2019,2529,4,1148,21,306,58,48,293,...,5089,183,36,505,43,4,6147,64,135,17
18,AZ,2021,7265,54,9257,319,2379,175,220,1939,...,12190,2536,348,3947,374,104,13542,3545,619,206
222,VT,2018,923,3,775,46,200,11,8,127,...,224,88,23,227,15,8,340,195,96,30
136,NC,2020,20341,117,35529,1033,5269,472,371,4428,...,22593,3978,825,8853,555,148,14141,10485,2455,699


In [3]:
#Check missing values
relationships.isnull().sum(axis=0)

Abbreviation                     0
Year                             0
Acquaintance                     0
Babysittee                       0
Boyfriend/Girlfriend             0
Child of Boyfriend/Girlfriend    0
Child                            0
Employee                         0
Employer                         0
Friend                           0
Grandchild                       0
Grandparent                      0
Homosexual Relationship          0
In-Law                           0
Neighbor                         0
Other Family Member              0
Otherwise Known                  0
Parent                           0
Relationship Unknown             0
Sibling                          0
Stepchild                        0
Spouse                           0
Stepparent                       0
Stepsibling                      0
Stranger                         0
Offender                         0
Ex Spouse                        0
Common Law Spouse                0
dtype: int64

In [4]:
relationships.shape

(247, 28)

In [5]:
relationships.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Abbreviation                   247 non-null    object
 1   Year                           247 non-null    int64 
 2   Acquaintance                   247 non-null    int64 
 3   Babysittee                     247 non-null    int64 
 4   Boyfriend/Girlfriend           247 non-null    int64 
 5   Child of Boyfriend/Girlfriend  247 non-null    int64 
 6   Child                          247 non-null    int64 
 7   Employee                       247 non-null    int64 
 8   Employer                       247 non-null    int64 
 9   Friend                         247 non-null    int64 
 10  Grandchild                     247 non-null    int64 
 11  Grandparent                    247 non-null    int64 
 12  Homosexual Relationship        247 non-null    int64 
 13  In-La

We found that the FBI relationships data does not have any missing values.  
Moreover, we further checked the dataset and confirmed that there were no empty data that replaced by "Null" or "999999".

### COVID-19 Stay-At-Home Order Data

In [6]:
#load in csv file
stayathome = pd.read_csv('COVID-19 US state policy database (CUSP) - Stay at Home.csv')
#drop irrelevant columns
stayathome = stayathome.drop(['Unnamed: 7', 'Notes', 'Religious Gatherings Exempt Without Clear Social Distance Mandate*', 'Stay-at-home order issued but did not specifically restrict movement of the general public'], axis=1)
#drop irrelavant row (this is just the totals of all the other columns)
stayathome = stayathome.drop([51])
#view data
stayathome.sample(5)

,State,State Abbreviation,State FIPS Code,Stay at home/shelter in place,End stay at home/shelter in place
7,Delaware,DE,10.0,3/24/2020,6/1/2020
24,Mississippi,MS,28.0,4/3/2020,4/27/2020
48,West Virginia,WV,54.0,3/24/2020,5/5/2020
22,Michigan,MI,26.0,3/24/2020,6/1/2020
17,Kentucky,KY,21.0,0,6/29/2020


In [7]:
#Check missing values
stayathome.isnull().sum(axis=0)

State                                 0
State Abbreviation                    0
State FIPS Code                       0
Stay at home/shelter in place         0
End stay at home/shelter in place     0
dtype: int64

There are no missing values to deal with in this datset, but upon further evaluation, it appears that the null values in the csv have been filled with 0s preemptively. This may cause an issue with certain columns (e.g. those that are datetime). We need to drop those columns in which Stay at home/shelter in place is 0, because this means that this state did not implement a shelter an place and will not help us understand how COVID-19 policies affected domestic violence.

In [8]:
# We dropped the states that did not conduct the stay-at-home order's 
no_shelter = stayathome[stayathome['Stay at home/shelter in place'] == '0'].index
stayathome = stayathome.drop(no_shelter)
stayathome.sample(5)

,State,State Abbreviation,State FIPS Code,Stay at home/shelter in place,End stay at home/shelter in place
10,Georgia,GA,13.0,4/3/2020,5/1/2020
42,Tennessee,TN,47.0,4/2/2020,4/29/2020
8,District of Columbia,DC,11.0,4/1/2020,5/29/2020
31,New Mexico,NM,35.0,3/24/2020,11/30/2020
12,Idaho,ID,16.0,3/25/2020,5/1/2020


In [9]:
#Check data types
stayathome.dtypes
#Change date columns to datetime
stayathome['Stay at home/shelter in place'] = pd.to_datetime(stayathome['Stay at home/shelter in place'], format = '%m/%d/%Y')
stayathome['End stay at home/shelter in place '] = pd.to_datetime(stayathome['End stay at home/shelter in place '], format = '%m/%d/%Y')
#Count stay at home policy days, create a new column for this
stayathome['Length of shelter in place'] = (stayathome['End stay at home/shelter in place '] - stayathome['Stay at home/shelter in place']).dt.days
# Change column names and add a year column for further merge
stayathome.rename(columns={'State Abbreviation': 'Abbreviation',}, inplace=True)
stayathome['Year']=2020
stayathome.sample(5)

,State,Abbreviation,State FIPS Code,Stay at home/shelter in place,End stay at home/shelter in place,Length of shelter in place,Year
8,District of Columbia,DC,11.0,2020-04-01,2020-05-29,58,2020
30,New Jersey,NJ,34.0,2020-03-21,2020-06-09,80,2020
28,Nevada,NV,32.0,2020-03-31,2020-05-09,39,2020
4,California,CA,6.0,2020-03-19,2021-01-25,312,2020
35,Ohio,OH,39.0,2020-03-24,2020-05-20,57,2020


In [10]:
stayathome.shape

(40, 7)

In [11]:
stayathome.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 49
Data columns (total 7 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   State                               40 non-null     object        
 1   Abbreviation                        40 non-null     object        
 2   State FIPS Code                     40 non-null     float64       
 3   Stay at home/shelter in place       40 non-null     datetime64[ns]
 4   End stay at home/shelter in place   40 non-null     datetime64[ns]
 5   Length of shelter in place          40 non-null     int64         
 6   Year                                40 non-null     int64         
dtypes: datetime64[ns](2), float64(1), int64(2), object(2)
memory usage: 2.5+ KB


### Socioeconomic Data: GDP Data and Personal Income Data

In [12]:
# Load the data and change column names for further merging
gdp = pd.read_csv('GDP_per_state_table.csv')
gdp.rename(columns={'GeoName': 'State',}, inplace=True)
income = pd.read_csv('personal_income_data.csv')
income.rename(columns={'GeoName': 'State'}, inplace=True)

In [13]:
# Add a abbreviation columns for further merging
states_data = [
    ("Alabama", "AL"), ("Alaska", "AK"), ("Arizona", "AZ"), ("Arkansas", "AR"), ("California", "CA"), ("Colorado", "CO"),
    ("Connecticut", "CT"), ("Delaware", "DE"), ("Florida", "FL"), ("Georgia", "GA"), ("Hawaii", "HI"),("Idaho", "ID"),("Illinois", "IL"),
    ("Indiana", "IN"), ("Iowa", "IA"), ("Kansas", "KS"), ("Kentucky", "KY"), ("Louisiana", "LA"), ("Maine", "ME"), ("Maryland", "MD"),
    ("Massachusetts", "MA"),("Michigan", "MI"), ("Minnesota", "MN"),("Mississippi", "MS"), ("Missouri", "MO"), ("Montana", "MT"),
    ("Nebraska", "NE"), ("Nevada", "NV"), ("New Hampshire", "NH"), ("New Jersey", "NJ"), ("New Mexico", "NM"), ("New York", "NY"),
    ("North Carolina", "NC"),("North Dakota", "ND"), ("Ohio", "OH"),("Oklahoma", "OK"), ("Oregon", "OR"),("Pennsylvania", "PA"),
    ("Rhode Island", "RI"),("South Carolina", "SC"),("South Dakota", "SD"), ("Tennessee", "TN"), ("Texas", "TX"), ("Utah", "UT"),
    ("Vermont", "VT"), ("Virginia", "VA"), ("Washington", "WA"), ("West Virginia", "WV"),("Wisconsin", "WI"), ("Wyoming", "WY")
]
states_df = pd.DataFrame(states_data, columns=['State', 'Abbreviation'])

In [14]:
# Merge the GDP and Personal income data for further merging
gdp=pd.melt(gdp,id_vars=['GeoFips','State'],var_name='Year',value_name='GDP')
income=pd.melt(income,id_vars=['GeoFips','State'],var_name='Year',value_name='Personal Income')
income= income.drop(['State'], axis=1)

In [15]:
gdp.sample(5)

,GeoFips,State,Year,GDP
249,11000,District of Columbia,2022,144029.6
258,21000,Kentucky,2022,217568.1
132,15000,Hawaii,2020,79592.5
115,94000,Plains,2019,1287230.5
131,13000,Georgia,2020,602321.8


In [16]:
income.sample(5)

,GeoFips,Year,Personal Income
242,37000,2022,621705.6
107,4000,2020,374311.8
185,32000,2021,192006.1
60,10000,2019,52740.7
70,21000,2019,195419.7


## 2. Merge Data
- FBI Crime Relational Data
- Socioeconomic Data: GDP Data and Personal Income Data
- COVID-19 Stay-At-Home Order Data

### Merge Relational Data & Socioeconomic Data

In [17]:
# merge the gdp dataframe and the income data frame together, created a socioeconomic dataframe for further merging
socioeconomic=gdp.merge(income,on=['GeoFips','Year'],how='inner')
socioeconomic=socioeconomic.merge(states_df,on='State',how='inner')
socioeconomic.sample(5)

,GeoFips,State,Year,GDP,Personal Income,Abbreviation
102,25000,Massachusetts,2020,556196.4,541420.5,MA
242,55000,Wisconsin,2020,322972.7,326836.5,WI
83,21000,Kentucky,2021,214565.1,232362.6,KY
22,6000,California,2020,2925146.9,2767521.4,CA
112,27000,Minnesota,2020,358127.2,349887.4,MN


In [18]:
socioeconomic.shape

(250, 6)

In [19]:
socioeconomic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   GeoFips          250 non-null    int64  
 1   State            250 non-null    object 
 2   Year             250 non-null    object 
 3   GDP              250 non-null    float64
 4   Personal Income  250 non-null    float64
 5   Abbreviation     250 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 13.7+ KB


In [20]:
# merge the relationship data and the socioeconomic data together
socioeconomic['Year']=socioeconomic['Year'].astype('int64')
relationships = relationships.merge(socioeconomic, on=['Abbreviation','Year'],how='inner')
relationships.sample(5)

,Abbreviation,Year,Acquaintance,Babysittee,Boyfriend/Girlfriend,Child of Boyfriend/Girlfriend,Child,Employee,Employer,Friend,...,Stepparent,Stepsibling,Stranger,Offender,Ex Spouse,Common Law Spouse,GeoFips,State,GDP,Personal Income
35,DE,2018,3237,8,3164,123,726,29,22,476,...,98,52,3040,2335,297,24,10000,Delaware,70856.4,51150.4
13,AR,2021,7862,25,11520,266,2106,169,135,1536,...,296,59,2734,3693,1301,108,5000,Arkansas,135568.8,156361.5
85,LA,2018,2228,4,3254,71,487,28,34,454,...,90,26,597,1495,216,221,22000,Louisiana,245220.3,212320.8
23,CA,2021,7655,47,11078,1419,4219,284,214,2054,...,258,97,17895,2788,751,381,6000,California,3146185.3,3013676.9
159,NY,2020,82,0,3969,2,17,3,9,19,...,2,2,187,0,3830,0,36000,New York,1650567.2,1405028.8


In [21]:
socioeconomic.shape

(250, 6)

In [22]:
socioeconomic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   GeoFips          250 non-null    int64  
 1   State            250 non-null    object 
 2   Year             250 non-null    int64  
 3   GDP              250 non-null    float64
 4   Personal Income  250 non-null    float64
 5   Abbreviation     250 non-null    object 
dtypes: float64(2), int64(2), object(2)
memory usage: 13.7+ KB


### Merge Relational + Socioeconomic Data & Stay-at-Home Order data

In [23]:
#Merge Relational + Socioeconomic Data & Stay-at-Home Order data
stayathome_merge=stayathome[['Abbreviation','Year','Length of shelter in place']]
stayathome_relationships = relationships.merge(stayathome_merge, on=['Abbreviation','Year'],how='left')
stayathome_relationships.sample(5)

,Abbreviation,Year,Acquaintance,Babysittee,Boyfriend/Girlfriend,Child of Boyfriend/Girlfriend,Child,Employee,Employer,Friend,...,Stepsibling,Stranger,Offender,Ex Spouse,Common Law Spouse,GeoFips,State,GDP,Personal Income,Length of shelter in place
96,MD,2019,1400,10,1029,302,259,72,26,239,...,11,2713,857,36,13,24000,Maryland,402346.2,380342.1,NaN
146,NJ,2022,5667,18,11935,183,966,355,238,1529,...,35,13572,1255,628,267,34000,New Jersey,646731.3,714990.2,NaN
29,CO,2022,12430,47,14056,297,2834,351,411,3038,...,102,32648,4461,1523,714,8000,Colorado,416114.4,442213.0,NaN
202,TX,2018,25611,123,30050,437,5136,746,456,4091,...,249,23103,7457,3927,4822,48000,Texas,1746543.3,1463789.8,NaN
241,WY,2022,1855,14,955,34,351,28,32,327,...,10,1185,795,102,24,56000,Wyoming,37294.2,42584.7,NaN


In [24]:
# Change the NaN data as 0, for COVID-19 did not happen in those years.
stayathome_relationships['Length of shelter in place'].fillna(0, inplace=True)
stayathome_relationships.sample(5)

,Abbreviation,Year,Acquaintance,Babysittee,Boyfriend/Girlfriend,Child of Boyfriend/Girlfriend,Child,Employee,Employer,Friend,...,Stepsibling,Stranger,Offender,Ex Spouse,Common Law Spouse,GeoFips,State,GDP,Personal Income,Length of shelter in place
133,NC,2022,22571,150,32918,978,5793,639,526,4259,...,157,13962,12380,2121,713,37000,North Carolina,609058.2,621705.6,0.0
179,PA,2020,393,1,250,11,81,6,8,23,...,1,295,2,17,5,42000,Pennsylvania,736470.6,783825.6,65.0
114,MN,2022,8761,49,7616,309,2315,135,72,862,...,78,5415,2590,469,81,27000,Minnesota,379111.5,393569.0,0.0
90,MA,2018,7313,57,12910,237,1871,200,160,1879,...,88,10424,1702,1010,101,25000,Massachusetts,549262.1,483954.4,0.0
142,NH,2021,3324,17,2949,75,776,99,102,993,...,48,4657,855,271,30,33000,New Hampshire,89890.1,100196.8,0.0


In [25]:
stayathome_relationships.shape

(242, 33)

In [26]:
stayathome_relationships.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242 entries, 0 to 241
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Abbreviation                   242 non-null    object 
 1   Year                           242 non-null    int64  
 2   Acquaintance                   242 non-null    int64  
 3   Babysittee                     242 non-null    int64  
 4   Boyfriend/Girlfriend           242 non-null    int64  
 5   Child of Boyfriend/Girlfriend  242 non-null    int64  
 6   Child                          242 non-null    int64  
 7   Employee                       242 non-null    int64  
 8   Employer                       242 non-null    int64  
 9   Friend                         242 non-null    int64  
 10  Grandchild                     242 non-null    int64  
 11  Grandparent                    242 non-null    int64  
 12  Homosexual Relationship        242 non-null    int

## 3. Data Calculation

In [27]:
# Calculate the total crime and the proportion of the Private Conflict
stayathome_relationships['Total Crime'] = stayathome_relationships[['Acquaintance', 'Babysittee', 'Boyfriend/Girlfriend', 'Child of Boyfriend/Girlfriend',
                                              'Child','Employee', 'Employer', 'Friend', 'Grandchild', 'Grandparent', 'Homosexual Relationship',
                                              'In-Law', 'Neighbor', 'Other Family Member','Otherwise Known', 'Parent',
                                              'Relationship Unknown', 'Sibling', 'Stepchild', 'Spouse', 'Stepparent', 'Stepsibling', 'Stranger',
                                             'Offender', 'Ex Spouse', 'Common Law Spouse',]].sum(axis=1)
stayathome_relationships['Private Conflict']=stayathome_relationships[['Babysittee', 'Child','Grandchild', 'Grandparent',
                                              'In-Law', 'Neighbor', 'Other Family Member', 'Parent','Sibling', 'Stepchild', 'Spouse', 'Stepparent',
                                             'Ex Spouse', 'Common Law Spouse',]].sum(axis=1)
stayathome_relationships['Private Conflict Rate']=stayathome_relationships['Private Conflict']/stayathome_relationships['Total Crime'] 

In [28]:
stayathome_relationships.shape

(242, 36)

In [29]:
stayathome_relationships.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242 entries, 0 to 241
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Abbreviation                   242 non-null    object 
 1   Year                           242 non-null    int64  
 2   Acquaintance                   242 non-null    int64  
 3   Babysittee                     242 non-null    int64  
 4   Boyfriend/Girlfriend           242 non-null    int64  
 5   Child of Boyfriend/Girlfriend  242 non-null    int64  
 6   Child                          242 non-null    int64  
 7   Employee                       242 non-null    int64  
 8   Employer                       242 non-null    int64  
 9   Friend                         242 non-null    int64  
 10  Grandchild                     242 non-null    int64  
 11  Grandparent                    242 non-null    int64  
 12  Homosexual Relationship        242 non-null    int

In [30]:
#we saved our dataset to a csv file for the visualization part of our project
stayathome_relationships.to_csv('final_dataset.csv')

### Reference: 
- Lecture Material: Lecture3_DataWrangling I
- Lecture Material: Lecture3_DataWrangling II
- Lecture Material: TimeSeriesData_F23_YourNameHere
- https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries
- https://towardsdatascience.com/all-about-missing-data-handling-b94b8b5d2184
- https://pandas.pydata.org/docs/reference/api/pandas.pivot.html
- https://pandas.pydata.org/docs/reference/api/pandas.melt.html#pandas.melt
- https://pandas.pydata.org/docs/reference/groupby.html
- https://pandas.pydata.org/docs/reference/arrays.html
- https://www.statology.org/pandas-set-first-row-as-header/
- https://plotly.com/python/choropleth-maps/
- https://www.educative.io/answers/how-to-create-a-choropleth-map-with-plotly-express-in-python
- https://www.statology.org/pandas-mean-by-group/
- https://www.programiz.com/python-programming/nested-dictionary